# Reading Data

TensorFlow程序获取数据的方法主要有三种：
* Feeding：Python代码提供数据
* Reading from files: 从文件读取数据
* Preloaded data: TensorFlow计算图中用constant或variable保存数据（只能是小数据集）

## Feeding

TensorFlow的Feeding机制允许你为计算图中任意Tensor注入数据。Python中可以直接为图节点提供数据。

为run()或eval()的`feed_dict`参数提供数据。

In [ ]:
with tf.Session():
    input = tf.placeholder(tf.float32)
    classifier = ...
    print(classifier.eval(feed_dict={input: my_python_preprocessing_fn()}))

你可以为任何Tensor进行feed数据，也包括constant和variable，我们建议还是为那些`tf.placholder`节点进行feed data。可以这么说，**placeholder存在的意义就是作为feed的目标**。如果你忘了给placeholder feed data，就执行程序，是会报错滴。




## Reading from files 

如果要从文件读取数据，pipeline通常包括下面的步骤:

* 1. 文件名列表
* 2. （可选的）文件名shuffle
* 3. (可选的) epoch limit
* 4. 文件名队列
* 5. Reader
* 6. 解码器，解码reader读取的record
* 7. （可选的）预处理
* 8. 实力队列

### 文件名、shuffling、epoch limit

对于文件名列表，可以用一个string Tensor表示,["file0", "file1"]或[("fiel%d" %i) for i in range(2)]；也可以用`tf.train.match_filenames_once`方法。



把文件名列表传给`tf.train.string_input_producer`方法。它会创建一个队列来保存文件名。



#### 文件格式

**CSV文件**

使用`tf.TextLineReader`和`tf.decode_csv`读取csv文件：

In [1]:
import tensorflow as tf
filename_queue = tf.train.string_input_producer(["file0.csv", "file1.csv"], num_epochs=1, shuffle=False)

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# 默认值
record_defaults = [[1], [1], [1], [1], [1]]
col1, col2, col3, col4, col5 = tf.decode_csv(
        value, record_defaults=record_defaults)
features = tf.stack([col1, col2, col3, col4])

with tf.Session() as session:
    # Start populating the filename queue
    session.run(tf.global_variables_initializer())
    session.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    ct = 1
    
    while True:
        try:
            example, label = session.run([features, col5])
            print(example, label, ct)
            ct += 1
        except tf.errors.OutOfRangeError:
            break
        
    coord.request_stop()
    coord.join(threads)
    



(array([1, 1, 1, 1], dtype=int32), 4, 1)
(array([2, 2, 2, 2], dtype=int32), 5, 2)
(array([1, 2, 1, 2], dtype=int32), 5, 3)
(array([2, 1, 2, 1], dtype=int32), 6, 4)
(array([1, 1, 1, 1], dtype=int32), 5, 5)
(array([2, 2, 2, 2], dtype=int32), 6, 6)
